In [7]:
from data_utils.Ashare import *

# 以往数据测试
f = '30m' # '1m','5m','15m','30m','60m', 1d, 1w, 1M
timepoints = 10

A={}
A_scaled={}
A[1] = get_price('sz159934',frequency=f,count=timepoints) # 黄金
A[0] = get_price('sh511010',frequency=f,count=timepoints) # 债券
A[2] = get_price('sh513500',frequency=f,count=timepoints) # 标普500
A[3] = get_price('sh000300',frequency=f,count=timepoints) # 沪深300
A[4] = get_price('sz399006',frequency=f,count=timepoints) # 创业板
A[5] = get_price('sz164824',frequency=f,count=timepoints) # 印度指数

print(A[1])
p = [0.4,0.2,0.1,0.1,0.1,0.1] # 资产比例

A_scaled[0] = A[0].div(A[0].iloc[0])
A_scaled[1] = A[1].div(A[1].iloc[0])
A_scaled[2] = A[2].div(A[2].iloc[0])
A_scaled[3] = A[3].div(A[3].iloc[0])
A_scaled[4] = A[4].div(A[4].iloc[0])
A_scaled[5] = A[5].div(A[5].iloc[0])

portfolio = sum(a * b for a, b in zip(A_scaled.values(), p))

# print(portfolio['close'])

asset_names = {
    0: "债券",
    1: "黄金",
    2: "标普500",
    3: "沪深300",
    4: "创业板",
    5: "印度指数"
}

# 提取每个资产的 close 列，并用中文名字重命名
close_dfs = [A[i]['close'].rename(asset_names[i]) for i in range(6)]
close_df = pd.concat(close_dfs, axis=1).round(6)
print(close_df)

                      open    high    low   close      volume
                                                             
2025-10-17 14:30:00  9.991   9.999  9.966   9.986  45827855.0
2025-10-17 15:00:00  9.986  10.003  9.951  10.003  55115891.0
2025-10-20 10:00:00  9.774   9.787  9.737   9.772  82948788.0
2025-10-20 10:30:00  9.772   9.782  9.766   9.769  36733976.0
2025-10-20 11:00:00  9.770   9.781  9.745   9.751  53824091.0
2025-10-20 11:30:00  9.751   9.767  9.744   9.754  39559474.0
2025-10-20 13:30:00  9.754   9.777  9.745   9.757  32093568.0
2025-10-20 14:00:00  9.758   9.764  9.715   9.716  55256963.0
2025-10-20 14:30:00  9.715   9.722  9.678   9.678  55483404.0
2025-10-20 15:00:00  9.679   9.700  9.668   9.685  52206078.0
                          债券      黄金  标普500     沪深300       创业板   印度指数
                                                                      
2025-10-17 14:30:00  140.230   9.986  2.275  4522.958  2947.356  1.511
2025-10-17 15:00:00  140.247  10.003  2.263

In [6]:
CCC = get_price('szau9999',frequency=f,count=timepoints) # 黄金
print(CCC)

Empty DataFrame
Columns: [open, high, low, close, volume]
Index: []


In [2]:
df = pd.read_csv("output.txt", sep=",")
last_row = df.tail(1)

# 今天日期（取 close_df 的最后一天作为基准）
now = close_df.index[-2]

# 检查是否已经有今天的数据
last_date = pd.to_datetime(last_row.iloc[0, 0])
if str(last_date) == str(now):
    print(f"✅ {now} 已经计算过，不再重复追加。")
else:
    # 取 close_df 最近两天
    prev_close = close_df.loc[last_date]
    today_close = close_df.loc[now]

    # 计算比例
    ratio = today_close / prev_close

    # 计算今天的资产（去掉第一列日期，只算资产部分）
    new_assets = last_row.iloc[0, 1:] * ratio.values

    # 拼接成新的 row
    new_row = pd.DataFrame([[now] + new_assets.round(2).tolist()],
                           columns=last_row.columns).round(2)

    # 追加到文件
    new_row.to_csv("output.txt", sep=",", header=False, index=False, mode="a")

    print("✅ 已添加新的一行：")
    print(new_row)

✅ 已添加新的一行：
                   时间        债券       黄金    标普500   沪深300      创业板     印度指数
0 2025-09-10 13:30:00  14108.17  7610.28  3703.12  3909.6  3889.27  3404.38


In [3]:
import numpy as np

df = pd.read_csv("output.txt", sep=",")
last_row = df.tail(1)
# 今天日期（取 close_df 的最后一天作为基准）
now = close_df.index[-1]

# 检查是否已经有今天的数据
last_date = pd.to_datetime(last_row.iloc[0, 0])

# 取 close_df 最近两天
prev_close = close_df.loc[last_date]
today_close = close_df.loc[now]

# 计算比例
ratio = today_close / prev_close

# 计算今天的资产（去掉第一列日期，只算资产部分）
new_assets = last_row.iloc[0, 1:] * ratio.values

# 拼接成新的 row
new_row = pd.DataFrame([[now] + new_assets.round(2).tolist()],
                        columns=last_row.columns).round(2)


assets = new_row.to_numpy()[0, 1:]

ratio = assets / np.sum(assets)
# ratio = np.floor(ratio*1e4)/1e4

diff_ratio = (ratio - p) / p
diff_ratio = np.floor(diff_ratio*1e4)/1e4

print(now)

# 转换为百分比
diff_percentage = diff_ratio * 100

# 打印每个资产的差距百分比
for idx, diff in enumerate(diff_percentage):
    print(f"{asset_names[idx]}: {diff:.2f}%")

# 找出差距超过20%的资产
exceed_idx = [idx for idx, diff in enumerate(diff_ratio) if np.abs(diff) > 0.2]

if exceed_idx:
    for idx in exceed_idx:
        print(f"{asset_names[idx]} 差距过大: 当前值={assets[idx]:.2f}, 目标值={np.sum(assets)*p[idx]:.2f}, 可行措施={assets[idx] - np.sum(assets)*p[idx]:.2f}")
else:
    print("✅ 所有资产均在合理范围内（偏差≤20%）")

2025-09-10 14:00:00
债券: -3.66%
黄金: 3.90%
标普500: 1.15%
沪深300: 6.71%
创业板: 6.04%
印度指数: -7.13%
✅ 所有资产均在合理范围内（偏差≤20%）
